# TMDB Test Extraction

- Extract Revenue, Budget and Certification for Movies Using API
- Years being requested: 2000 and 2001

### Data Source

The Movie Database (TMDB) - (https://www.themoviedb.org/)

Imports/Installaltions

In [1]:
#installations
!pip install tmdbsimple
!pip install tqdm

In [2]:
# Standard Imports
import numpy as np
import pandas as pd

# Additional Imports
#os for saving and loading files
#json - to work with json files
#time - to add a short pause to not overwhelm the server
import os, json, time

#python package
import tmdbsimple as tmdb

#progress bar
from tqdm.notebook import tqdm_notebook

### Functions

In [3]:
#function to add movie certification to the .info
def get_movie_with_rating(movie_id):
  """Adapted from source = https://github.com/celiao/tmdbsimple"""
  
  #get the movie object for the current id
  movie = tmdb.Movies(movie_id)

  #save the .info .releases dictinoaries
  movie_info = movie.info()
  releases = movie.releases()

  #loop through countries in releases to get only US
  for c in releases['countries']:
      #if the country abbreviation==US
      if c['iso_3166_1'] == 'US':
          #save a 'certification' key in the info with the certification
          movie_info['certification'] = c['certification']
  return movie_info

#Appends a list of records (new_data) to a json file (filename)
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""     
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [4]:
def read_and_fix_json(JSON_FILE):
    """Attempts to read in json file of records and fixes the final character
    to end with a ] if it errors.
    
    Args:
        JSON_FILE (str): filepath of JSON file
        
    Returns:
        DataFrame: the corrected data from the bad json file
    """
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    ## If read_json throws an error
    except:
        

        ## manually open the json file
        with open(JSON_FILE,'r+') as f:
            ## Read in the file as a STRING
            bad_json = f.read()
            
            ## if the final character doesn't match first, select the right bracket
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            ## Select expected final brakcet
            final_char = final_brackets[first_char]
            
            ## if the last character in file doen't match the first char, add it
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            ## Rewind to start of file and write new good_json to disk
            f.seek(0)
            f.write(good_json)
           
        ## Load the json file again now that its fixed
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df

### Uploading my API credentials for TMDB

In [5]:
#login credentials from json file
with open('/Users/Daisy/.secret/tmdb_api.json', 'r') as f: 
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [6]:
#setting API key variable 
tmdb.API_KEY =  login['api-key']

Uploading basics data

In [7]:
# Open basics dataset and preview
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama


### TEST Run to see if loop works

In [8]:
#loop to loop thru list of movie ids and save errors in dataframe
#test function on movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook")
test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28547,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10308,PG-13


### Create folder if doesn't exist

In [9]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

### Creating Loop

In [10]:
#years to extract from Loop
YEARS_TO_GET = [2001,2000]

#defining empty list to store errors
errors = [ ]

In [11]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    #if it does not exist: create it
    if file_exists == False:
    #save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
  
    #saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
  
    #saving movie ids to list
    movie_ids = df['tconst'].copy()

    #load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    #filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

        #INNER Loop - using functions to get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            #retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            #append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            #short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
        
        #saving errors in a list   
        except Exception as e:
            #continue
            errors.append([movie_id, e])

    #Finish OUTER Loop
    #Save the year's results as csv.gz files
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_movies_{YEAR}.csv.gz", compression="gzip", index=False)

#After loops finish print errors
print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1570 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1448 [00:00<?, ?it/s]

- Total errors: 455
